In [1]:
from PIL import Image
from pytesseract import image_to_string
import pytesseract

#### Change to False if you want your actual main stat value ####
assume_max_lv_gear = True

#### Screenshot path ####
sh_path = 'screenshots/'

def process(k,img):
    if  k == 'plus' or k == 'level':
        #since level and plus are read off the picture of the gear, there is a lot of noise
        #so we need an iterative process to do it
        thresh = cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU
        low = 81
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
        if not any(i.isdigit() for i in data):
            low = 100
            proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
            data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
            if not any(i.isdigit() for i in data):
                low = 125
                proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
                data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
                if not any(i.isdigit() for i in data): data = str(0)
                  
    else:
        thresh = cv2.THRESH_BINARY_INV
        low = 70
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_BGR2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 6').replace('Rina','Ring').replace('Edic','Epic').replace('Enic','Epic')
    return data

def stat_converter(stat):
    result = ''
    if 'attack' in stat.lower():
        result = 'Atk'
        if '%' in stat: result += 'P'
    if 'health' in stat.lower():
        result = 'HP'
        if '%' in stat: result += 'P'
    if 'defense' in stat.lower():
        result = 'Def'
        if '%' in stat: result += 'P'
    if 'speed' in stat.lower(): result = 'Spd'
    if 'chance' in stat.lower(): result = 'CChance'
    if 'damage' in stat.lower(): result = 'CDmg'
    if 'effectiveness' in stat.lower(): result = 'Eff'
    if 'resistance' in stat.lower(): result = 'Res'
    return result
  

def digit_filter(val):
    if not val: 
        return 0
    if val is '.':
        return 0
    else:
        return int(''.join(filter(str.isdigit,val)))

def char_filter(val):
    return ''.join(filter(str.isalpha,val)).capitalize()
  
def max_stat(data,item):
    stat = stat_converter(data)
    val = digit_filter(data) # Begin by setting val = actual value
    if item['ability'] < 15: # Only change stats on items where they need to be increased
        if item['level'] in range(1,15):
            if stat == 'CChance': val = 5
            elif stat == 'CDmg': val = 15
            elif stat == 'Spd': val = 15
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 10
        elif item['level'] in range(15,30):
            if stat == 'CChance': val = 15
            elif stat == 'CDmg': val = 25
            elif stat == 'Spd': val = 20
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 20
        elif item['level'] in range(30,44):
            if stat == 'CChance': val = 25
            elif stat == 'CDmg': val = 35
            elif stat == 'Spd': val = 25
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 30
        elif item['level'] in range(44,58):
            if stat == 'CChance': val = 35
            elif stat == 'CDmg': val = 45
            elif stat == 'Spd': val = 30
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 40
        elif item['level'] in range(58,72):
            if stat == 'CChance': val = 45
            elif stat == 'CDmg': val = 55
            elif stat == 'Spd': val = 35
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 50
            elif stat == 'HP': val = 2360
            elif stat == 'Def': val = 260
            elif stat == 'Atk': val = 440
        elif item['level'] in range(72,86):
            if stat == 'CChance': val = 55
            elif stat == 'CDmg': val = 65
            elif stat == 'Spd': val = 40
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 60
            elif stat == 'HP': val = 2700
            elif stat == 'Def': val = 300
            elif stat == 'Atk': val = 500
        elif item['level'] in range(86,89):
            if stat == 'CChance': val = 60
            elif stat == 'CDmg': val = 70
            elif stat == 'Spd': val = 45
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 65
            elif stat == 'HP': val = 2765
            elif stat == 'Def': val = 310
            elif stat == 'Atk': val = 515
    return val
            

In [2]:
from glob import glob
from string import ascii_lowercase, digits
import random
import json
import cv2
import pytesseract

#format for optimizer
export = {'processVersion':'1','heroes':[],'items':[]}

temp_list = []
print('Beginning export process...')

#iterate through the screenshots
filenames = glob(sh_path+'*.png')
filenames.extend(glob(sh_path+'*.jpg'))

#print(filenames)
for n,name in enumerate(filenames):
    #because the height of the item box changes depending on the length of the item and set descriptions,
    #we have to crop the top and bottom info separately in order to ensure the OCR boxes within these areas
    #remain in fixed locations. we then process the top and bottom info independently.
    
    img = cv2.imread(name)
    #print(name)
    #print(img.shape)
    
    #top box
    temp_top = cv2.imread('e7/top.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_top, cv2.TM_CCOEFF_NORMED))
    top_box = img[max_loc[1]:max_loc[1]+160,740:1190] # Fixed width, then crop 160 pixels from top triangle

    #bottom box
    temp_bot = cv2.imread('e7/bottom.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_bot, cv2.TM_CCOEFF_NORMED))
    bottom_box = img[max_loc[1]+25:max_loc[1]+360,740:1190] # Fixed width, shift down 25 from divider, then crop 335 pixels deep

    #setup item dictionary
    item = {'locked':False,'efficiency':0}

    #process top image
    top_coords = {'type': [[20,70],[172,432]],
                  'level': [[19,44],[37,66]],
                  'plus': [[11,34],[139,168]]}
    
    print('currently processing',name)
    
    for k in top_coords.keys():
        data = process(k,top_box[top_coords[k][0][0]:top_coords[k][0][1],top_coords[k][1][0]:top_coords[k][1][1]])
        if k == 'type':
            item['rarity'] = char_filter(data.split(' ')[0])
            item['slot'] = char_filter(data.split(' ')[1].split('\n')[0])
        if k == 'level':
            item['level'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))
        if k == 'plus':
            item['ability'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))

    print(name, "shows lvl", item['level'],"and enhance lv",item['ability'])
    #process bottom image
    bot_coords = {'main': [[8,70],[65,435]],
                  'subs': [[98,255],[25,435]],
                  'set': [[280,340],[76,435]]}
    
    for k in bot_coords.keys():
        data = process(k,bottom_box[bot_coords[k][0][0]:bot_coords[k][0][1],bot_coords[k][1][0]:bot_coords[k][1][1]])
        if k == 'main':
            #print(data)
            stat = stat_converter(data)
            if assume_max_lv_gear is True: item['mainStat'] = [stat,max_stat(data,item)]
            else: item['mainStat'] = [stat,digit_filter(data)]     
        
        if k == 'subs':
            #print(data.split('\n'))
            for n,entry in enumerate(data.split('\n')):
                stat = stat_converter(entry)
                val = digit_filter(entry.replace('T','7'))
                item['subStat'+str(n+1)] = [stat,val]
        if k == 'set':
            #print(data)
            item['set'] = char_filter(data.split(' Set')[0])
    
    # A common OCR error is to fail to read the level. If this happens, we need to set it to 1 so it will import properly.
    if item['level'] == 0: item['level'] = 1
    #print(item)
    
    #checks to see if there are duplicates, and if not, give item unique ID and place in 'export'
    if item not in temp_list:
        temp_list.append(item)
        #print(export['items'])
        item['id'] = 'jt'+''.join(random.choice(digits+ascii_lowercase) for _ in range(6))
        export['items'].append(item)
    
    # outputs progress of export
    print(len(export['items']),"exported out of",len(filenames))
    
    
#export to json for importing into optimizer
with open('exported_gear.json', 'w') as f: json.dump(export, f)
print('JSON file finished!')

Beginning export process...
currently processing screenshots/Screenshot_20191223-061251.png
screenshots/Screenshot_20191223-061251.png shows lvl 88 and enhance lv 15
1 exported out of 6
currently processing screenshots/Screenshot_20191223-061244.png
screenshots/Screenshot_20191223-061244.png shows lvl 88 and enhance lv 15
2 exported out of 6
currently processing screenshots/Screenshot_20191223-061227.png
screenshots/Screenshot_20191223-061227.png shows lvl 85 and enhance lv 15
3 exported out of 6
currently processing screenshots/Screenshot_20191223-061233.png
screenshots/Screenshot_20191223-061233.png shows lvl 85 and enhance lv 15
4 exported out of 6
currently processing screenshots/Screenshot_20191223-061213.png
screenshots/Screenshot_20191223-061213.png shows lvl 85 and enhance lv 15
5 exported out of 6
currently processing screenshots/Screenshot_20191223-061239.png
screenshots/Screenshot_20191223-061239.png shows lvl 88 and enhance lv 15
6 exported out of 6
JSON file finished!
